In [85]:
import bz2
import xml.sax as sax
import xml.etree.cElementTree as ET
import itertools as it
import re

In [31]:
FILE_BZ = 'fiwiktionary-20180301-pages-articles-multistream.xml.bz2'
FILE = 'fiwiktionary-20180301-pages-articles-multistream.xml'
NS = '{http://www.mediawiki.org/xml/export-0.10/}'

In [32]:
def tag(tag_name):
    return NS + tag_name

In [61]:
def text(elem):
    return "".join(elem.itertext())
    

In [73]:
SYNONYM_LABEL = '=====Synonyymit====='

In [105]:
def extract_synonyms(txt):
    syns_l = it.dropwhile(lambda l: l != SYNONYM_LABEL, txt.split('\n'))
    syns_l = list(syns_l)[1:]
    syns_l = it.takewhile(lambda l: l.startswith('*'), syns_l)
    print(list(syns_l))
    syns_l = map(lambda l: re.sub(r"''\(\w+\)''", '', l), syns_l)
    syns_l = [t for l in syns_l for t in l.split(' ')[2:]]
    syns_l = [re.sub(r"\[\[|\]\]", '', t) for t in syns_l]
    return list(syns_l)

all_syns = {}
    
for event, elem in ET.iterparse(bz2.open(FILE_BZ), events=['end']):
    if event == 'end':
        if elem.tag == tag('page'):
            ns_el = elem.find(tag('ns'))
            if text(ns_el) == '0':
                title_el = elem.find(tag('title'))
                title = text(title_el)
                print(title)
                revision_el = elem.find(tag('revision'))
                text_el = revision_el.find(tag('text'))
                txt = text(text_el)
                syns = extract_synonyms(txt)
                all_syns[title] = syns
                if len(all_syns) == 2:
                    break
all_syns

haarukka
["* ''(murteellinen)'' [[kahveli]]"]
Suomi
['*{{vanha}} [[Suomenmaa]]', '*{{kuva}} [[Tuhansien järvien maa]]', '*{{arkikieli}} [[Kekkoslovakia]]', '*{{halventava}} [[Kieltola]]', '*{{halventava}} [[Hölmölä]]', '*{{halventava}} [[Byroslavia]]']


{'Suomi': [], 'haarukka': []}

In [103]:
len(all_syns)

86568